The most recent version of this notebook is available at https://github.com/nadiinchi/dl_labs/blob/master/lab_attention.ipynb

Этот ноутбук содержит практическое введение в механизмы внимания.
В ноутбуке рассматривается модельная задача и игрушечная архитектура, поэтому он может быть полностью выполнен на CPU.

Рассматриваемая задача - перемножение перестановок.
Длина перестановок фиксирована и равна perm_len.
Вход - целочисленный вектор длины 2 x perm_len - представляет собой две последовательно записанные перестановки p1 и p2.
Произведением p1 и p2 считается такая перестановка p3, что p3[i] = p1[p2[i]].
Требуемый выход сети - также целочисленный вектор длины 2 x perm_len, первые perm_len элементов которого равны нулю, а последние являются перестановкой p3.

Пример для perm_len = 5:
```
Вход сети:  3 4 2 1 0 1 3 0 2 4
Выход сети: 0 0 0 0 0 4 1 3 2 0
Пояснение:  p1 = 3 4 2 1 0,    p2 = 1 3 0 2 4   =>    p3 = 4 1 3 2 0
```

Теоретически такая задача может быть решена обычным LSTM, который сначала запомнит в скрытом состоянии перестановку p1, а затем проходя по перестановке p2 будет выдавать соответствующие элементы из перестановки p1.
На практике, однако, такая модель работает заметно хуже модели с вниманием. Модель с вниманием в явном виде учится проходя по перестановке p2 обращать внимание на нужный элемент перестановки p1 и выдавать его.

В задании требуется реализовать и сравнить различные виды внимания, используемые в реальных задачах.
Также следует реализовать и использовать кодирование позиций, описанное в статье о трансформерах.
Подробнее эти слои и модель описаны ниже.

In [ ]:
import torch
from torch import nn
from torch import optim
import numpy as np
import math

Ниже предлагается реализовать несколько моделей внимания, описанных в разных статьях.
В общем случае есть $K$ объектов, на которые можно обращать внимание.
Каждый объект характеризуется ключом $k_i$ и значением $v_i$.
К слою внимания приходят запросы.
Для запроса $q$ слой возвращает взвешенную сумму значений объектов, с весами, пропорциональными степени соответствия ключа запросу:
$$w_i = \frac{\exp(score(q, k_i))}{\sum_{j=1}^K\exp(score(q, k_j))}$$
$$a = \sum_{i=1}^K w_i v_i$$

Почти всегда запросы, ключи и значения - вещественные векторы некоторых фиксированных размерностей.
В задании предлагается реализовать три вида внимания:
+ (опционально) Аддитивное внимание.
Задается функцией $score(q, k) = w_3^T \tanh ( W_1q + W_2k)$, где $W_1, W_2, w_3$ - обучаемые параметры слоя внимания.
Для такой функции запрос и ключ могут иметь разную размерность.
Матрицы $W_1$ и $W_2$ отображают запрос и ключ в общее скрытое пространство, размерность которого совпадает с размерностью вектора $w_3$.
Размерность скрытого пространства может быть выбрана произвольно и является гиперпараметром слоя.
Больше информации в статье Bahdanau et al. "Neural Machine Translation by Jointly Learning to Align and Translate", 2014.
+ Мультипликативное внимание.
Задается функцией $score(q, k) = q^Tk$.
Для применения такого типа внимания требуется чтобы размерность запроса совпадала с размерностью ключа.
Больше информации в статье Luong et al. "Effective approaches to attention-based neural machine translation", 2015.
+ Отмасштабированное мультипликативное внимание.
Задается функцией $score(q, k) = \frac{q^Tk}{\sqrt{dim(k)}}$, где $dim(k)$ - размерность ключа (также равная размерности запроса).
При обучаемых запросах или ключах такое внимание эквивалентно простому мультипликативному вниманию.
Однако сразу после инициализации такое внимание поощряет более сглаженные веса, что смягчает проблему маленьких градиентов при насыщении SoftMax.
Больше информации в статье Vaswani et al. "Attention Is All You Need", 2017.

На практике обычно используют архитектуру в которой ключи и значения объектов совпадают.
В прототипах снизу предлагается реализовать именно такую архитектуру.
Поскольку ключи и значения совпадают, то они передаются один раз и называются признаками объектов (features) (т. е. $f_i := k_i = v_i$).

Также для гибкости интерфейса и ускорения обучения все слои внимания ниже получают для каждого объекта батча по несколько запросов.

Класс Attention является родителем классов AdditiveAttention, MultiplicativeAttention и ScaledDotProductAttention.
Класс Attention  - абстрактный класс, то есть в качестве слоев  используются только наследники класса Attention, но никогда не он сам.

В классе Attention надо реализовать функцию attend, которая принимает для каждого элемента батча набор признаков объектов и набор запросов.
Функция attend использует функцию get_scores, реализованную во всех наследниках класса, затем используя полученные значения $score(q, f)$ вычисляет $w$ и $a$.

Mask - маска внимания, показывающая для каждого запроса, на какие объекты он не может обращать внимания.
Была предложена в статье Vaswani et al. "Attention Is All You Need", 2017.
Используется для того, чтобы обучаемая модель сохраняла авторегрессионные свойства, то есть чтобы выход слоя для $i$-ой позиции не зависел от значений входа последующих позиций.
В функции get_autoregressive_mask надо построить вышеописанную квадратную маску заданного размера.

Наиболее численно стабильным способом применения маски в attend будет обращение соответствующих значений score в -float('inf') до применения SoftMax.
Альтернативным способом является обнуление весов $w$ в соответствии с маской и их перенормировка, но такой способ менее вычислительно стабилен (подумайте, почему).

В каждом из классов AdditiveAttention, MultiplicativeAttention и ScaledDotProductAttention требуется реализовать функцию get_scores, которая для каждого элемента батча для каждого запроса возвращает его похожесть на объекты того же батча.
Код get_scores должен быть эквивалетнен следующему:
```
res = torch.zeros(batch_size, num_queries, num_objects)
for i in range(batch_size):
    for j in range(num_queries):
        for k in range(num_objects):
            res[i, j, k] = score(queries[i, j], features[i, k])
```
Естественно, вышеприведенный код служит лишь иллюстацией, объясняющей размерности аргументов и выхода функции get_scores.
Реализованный в задании код должен быть эффективно векторизован.

Подсказки по написанию кода:

+ В классе AdditiveAttention нужно завести обучаемые параметры $W_1$, $W_2$ и $w_3$.
  * Для желающих попрактиковаться в написании своих обучаемых слоев следует помнить, что тензор, содержащийся в наследнике nn.Module не будет перечислен в .parameters() от объекта этого класса, то есть по нему не будет производиться градиентный спуск.
Для того, чтобы тензор появился в .parameters(), надо обернуть его в nn.Parameter().
Также перед такой оберткой следует инициализировать его, используя какую-нибудь из стандартных инициализаций слоев pytorch.
Стоит обратить внимание, что инициализации в pytorch являются in-place, то есть сначала надо завести тензор, а потом передать его в функцию инициализации.
  * Для тех, кто не желает практиковаться в написании своих обучаемых слоев, можно вспомнить, что умножение на матрицу эквивалентно применению nn.Linear(.., bias=False).
аким образом, функцию внимания можно реализовать с помощью трех линейных слоев, соответствующих матрицам $W_1$, $W_2$, и $w_3$.
+ Рекомендуется обратить внимание на функцию torch.bmm, она может пригодиться во многих слоях внимания ниже.
+ Для визуализации карты внимания в конце ноутбука надо в функции attend сохранять weights.detach() в self.last_weights. .detach() используется для того, чтобы сохраненные веса не были частью графа вычилений. Не забывайте делать .detach() от отладочного вывода, чтобы граф вычислений не рос сверх необходимого размера.

In [ ]:
def get_autoregressive_mask(size):
    """
    Returns attention mask of given size for autoregressive model.
    """
    # your code here
    return res

In [ ]:
class Attention(nn.Module):
    def __init__(self):
        super().__init__()

    def get_scores(self, features, queries):
        """
        features: [batch_size x num_objects x obj_feature_dim]
        queries:  [batch_size x num_queries x query_feature_dim]
        Returns matrix of scores with shape [batch_size x num_queries x num_objects].
        """
        raise NotImplementedError()                

    def attend(self, features, queries, mask=None):
        """
        features:        [batch_size x num_objects x obj_feature_dim]
        queries:         [batch_size x num_queries x query_feature_dim]
        mask, optional:  [num_queries x num_objects]
        Returns matrix of features for queries with shape [batch_size x num_queries x obj_feature_dim].
        If mask is not None, sets corresponding to mask weights to zero.
        Saves detached weights as self.last_weights for further visualization.
        """
        # your code here
        return result

In [ ]:
class AdditiveAttention(Attention):
    """
    Bahdanau et al. "Neural Machine Translation by Jointly Learning to Align and Translate", 2014.
    """
    def __init__(self, obj_feature_dim, query_feature_dim, hidden_dim):
        """
        obj_feature_dim   - dimensionality of attention object features vector
        query_feature_dim - dimensionality of attention query vector
        hidden_dim        - dimensionality of latent vectors of attention 
        """
        super().__init__()
        # your code here

    def get_scores(self, features, queries):
        """
        features: [batch_size x num_objects x obj_feature_dim]
        queries:  [batch_size x num_queries x query_feature_dim]
        Returns matrix of scores with shape [batch_size x num_queries x num_objects].
        """
        # your code here
        return result

In [ ]:
class MultiplicativeAttention(Attention):
    """
    Luong et al. "Effective approaches to attention-based neural machine translation", 2015.
    """
    def __init__(self):
        super().__init__()

    def get_scores(self, features, queries):
        """
        features: [batch_size x num_objects x feature_dim]
        queries:  [batch_size x num_queries x feature_dim]
        Returns matrix of scores with shape [batch_size x num_queries x num_objects].
        """
        # your code here
        return result

In [ ]:
class ScaledDotProductAttention(Attention):
    """
    Vaswani et al. "Attention Is All You Need", 2017.
    """
    def __init__(self):
        super().__init__()

    def get_scores(self, features, queries):
        """
        features: [batch_size x num_objects x feature_dim]
        queries:  [batch_size x num_queries x feature_dim]
        Returns matrix of scores with shape [batch_size x num_queries x num_objects].
        """
        # your code here
        return result

In [ ]:
# time to check that your attention works
# your code here

Функция perm_generator генерирует батч заданного размера объектов для обучения или теста.
Для каждого объекта батча случайно равновероятно генерируются перестановки p1 и p2 длины perm_size.
Из них формируется из них входная последовательность [p1, p2] и корректный ответ [0, p3] для неё (см. пример выше).

In [ ]:
def perm_generator(batch_size, perm_size):
    """
    Generates batch of batch_size objects.
    Each object consists of two random permutations with length perm_size.
    The target for the object is the product of its two permutations.
    """
    # your code here
    return objects, correct_answers

In [ ]:
# time to check your generator
# your code here

PositionalEncoder - слой, описанный в Vaswani et al. "Attention Is All You Need", 2017.
Добавляет к выходу предыдущего слоя эмбеддинги позиций.
Чтобы не перевычсилять каждый раз эмбеддинги позиций, получает при создании параметр max_len и предподсчитывает эмбеддинги для позиций с 0 до max_len - 1 включительно.
Флаг add указывает, добавлять эмбединги позиций к выходу предыдущего слоя (по умолчанию, при add=True, используется в оригинальной статье) или конкатенировать (add=False).
Для выбранной размерности эмбеддингов следует визуализировать эмбеддинги (построить график каждой компоненты эмбеддингов) и подобрать подходящий параметр scale.

In [ ]:
class PositionalEncoder(nn.Module):
    def __init__(self, dim, max_len=50, scale=10000.0, add=True):
        """
        Transforms input as described by Vaswani et al. in "Attention Is All You Need", 2017.
        dim     - dimension of positional embeddings.
        max_len - maximal length of sequence, for precomputing
        scale   - scale factor for frequency for positional embeddings
        add     - boolean, if add is False, concatenate positional embeddings with input instead of adding
        """
        super().__init__()
        
        self.dim = dim
        self.add = add
        if add:
            self.extra_output_shape = 0
        else:
            self.extra_output_shape = dim

        # your code here
               
    def forward(self, input):
        """
        input - [batch_size x sequence_len x features_dim]
        If self.add is True, self.dim = featurs_dim.
        Returns input with added or concatenated positional embeddings (depending on self.add).
        """
        # your code here
        return result

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

# time to draw positional encoder
# your code here

Модель состоит последовательно из следующих слоев:
+ Эмбеддинг элементов входной последовательности.
+ Эмбеддинг позиций или None, обозначающее отсутствие этого слоя.
+ LSTM сеть.
Для вычсиления размерности входа LSTM сети можно исползовать размерность первого эмбеддинга и extra_output_shape для эмбеддинга позиций.
+ Слой внимания. В качестве запросов получает выходы lSTM сети, в качестве объектов - эмбеддинги последовательности.
При установленном флаге autoregressive использует маску внимания для того, чтобы не обращать внимания на элементы последовательности из будущего.
+ Логистическая регресиия с perm_len классами, выдающая для каждой позиции парвильный ответ.

Обратите внимание, что эта модель не является ни трансформером, ни традиционной сетью использующую LSTM с вниманием.
В трансформере используется K-головое внимание, поэлементное преобразование эмбеддингов.
В традиционный сетях с LSTM запрос, выданный сетью в предыдущий момент времени, влияет на вход сети в следующий момент времени, поэтому одновременная параллельная обработка всей последовательности невозможна.
Также в большинстве сетей используется архитектура энкодер-декодер, где сначала энкодер читает всю входную последовательность и формирует её скрытое представление, а затем декодер выдает выходную последовательность используя это скрытое представление и механизм внимания.

In [ ]:
class PermMultiplier(nn.Module):
    def __init__(self, perm_len, embedding_dim, hidden_dim, attention, pos_enc, autoregressive):
        """
        perm_len       - permutation length (the input is twice longer)
        embedding_dim  - dimensionality of integer embeddings
        hidden_dim     - dimensionality of LSTM output
        attention      - Attention object
        pos_enc        - PositionalEncoder object or None
        autoregressive - boolean, if True, then model must use autoregressive mask for attention
        """
        super().__init__()
        self.autoregressive = autoregressive
        self.perm_len = perm_len
        # your code here

    def forward(self, input):
        """
        Perform forward pass through layers:
        + get embeddings from input sequence (using both embeddings
          and positional embeddings if pos_enc is not None)
        + run LSTM on embeddings
        + use output of LSTM as an attention queries
        + attend on the embedded sequence using queries (note autoregressive flag)
        + make final linear tranformation to obtain logits
        """
        # your code here

Есть время писать модели и время обучать их.
Сейчас наступило второе.

Допишите код для обучения модели ниже.
Найдите правильную архитектуру и набор гиперпараметров для модели и метода оптимизации.
Утверждается, что для некоторой архитектуры и гиперпараметров модель обучается на CPU за небольшое время.

In [ ]:
perm_len = 10

In [ ]:
# time to set up a model
# you can check that without pos_enc model doesn't work
# not-autoregressive model can be learned easily, but it is less isefull
# try to learn autoregressive model if possible
pos_enc = PositionalEncoder(?, perm_len * 2, ?, ?)
attention = ?
model = PermMultiplier(perm_len, ?, ?, attention, pos_enc, ?)
if torch.cuda.is_available():
    model = model.cuda()

In [ ]:
# set up optimizer
gd = optim.Adam(model.parameters(), lr=?)

In [ ]:
# do optimization
avg_loss = None
forget = 0.99
batch_size = 64
iterator = range(?)
for i in iterator:
    gd.zero_grad()
    batch = perm_generator(batch_size, perm_len)
    if torch.cuda.is_available():
        batch = batch[0].cuda(), batch[1].cuda()
    # compute batch loss
    # your code here
    loss.backward()
    if avg_loss is None:
        avg_loss = float(loss)
    else:
        avg_loss = forget * avg_loss + (1 - forget) * float(loss)
    descr_str = 'Iteration %05d, loss %.5f.' % (i, avg_loss)
    print('\r', descr_str, end='')
    gd.step()

Отлично, у нас есть какая-то модель.
Давайте проверим, как она перемножает две случайных перестановки.

In [ ]:
# time to check your model
batch = perm_generator(batch_size, perm_len)
if torch.cuda.is_available():
    batch = batch[0].cuda(), batch[1].cuda()
print('Input:\n', batch[0][:5])
print('Output:\n', ?)
print('Correct:\n', batch[1][:5])

Одна из важных прикладных свойств внимания - отображать, на что обращает внимание модель.
Для этого используются так называемые карты внимания.
Используйте поле last_weights слоя Attention, чтобы визуализировать, на какие позиции в каждый момент времени обращала внимание обученная модель для перестановок из батча в ячейке выше.
Ожидаемое поведение - в каждый момент времени прохода по перестановке p2 обращать внимание на соответствующий элемент из перестановки p1.

In [ ]:
# visualize attention map for some object
# your code here

In [ ]:
# play with model and learn something new about attention!